# Análisis Cuantitativo para Tesis: Obras Públicas en Puno (2013-2025)

**Autor de la Tesis:** Bach. Fabricio Rigoberto Rodriguez Ascarrunz
**Análisis de Datos por:** Ing. Jhon Wilber Ajata Ascarrunz

**Objetivo:** Este notebook reproduce el análisis cuantitativo descrito en el Capítulo IV de la tesis. Se parte de la base de datos limpia de INFOBRAS para generar las tablas de resultados y realizar las pruebas de hipótesis que comparan las modalidades de ejecución por Administración Directa y por Contrata, siguiendo estrictamente la metodología y los indicadores definidos en el instrumento de investigación.

In [10]:
# Tarea 1.1: Importar librerías
import pandas as pd
import numpy as np
from scipy.stats import shapiro, mannwhitneyu

print("--- Paso 1: Entorno Configurado ---")
print("Librerías importadas correctamente.")

--- Paso 1: Entorno Configurado ---
Librerías importadas correctamente.


In [11]:
# Tarea 1.2: Cargar y preparar datos
print("\n--- Paso 2: Carga y Preparación de Datos ---")
try:
    # Cargamos el resultado de nuestro trabajo de limpieza anterior
    df_clean = pd.read_csv('../data/infobras_clean_v1.csv', low_memory=False)
    
    # Aseguramos que las columnas de fecha clave sean del tipo datetime para futuros análisis
    date_cols_to_convert = ['fecha_de_inicio_de_obra', 'fecha_de_finalizacion_real']
    for col in date_cols_to_convert:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
    
    print("Dataset 'infobras_clean_v1.csv' cargado y preparado.")
    
    # Filtramos para obtener solo los registros de la región Puno
    df_puno = df_clean[df_clean['departamento'] == 'PUNO'].copy()
    print(f"Se han aislado {len(df_puno)} obras correspondientes a la región Puno.")

except FileNotFoundError:
    print("Error Crítico: No se encontró el archivo 'infobras_clean_v1.csv' en la carpeta 'data/'.")
    print("Asegúrate de que el archivo exista en la ubicación correcta antes de continuar.")



--- Paso 2: Carga y Preparación de Datos ---
Dataset 'infobras_clean_v1.csv' cargado y preparado.
Se han aislado 10317 obras correspondientes a la región Puno.


In [13]:
# Obtenemos la lista de todas las columnas en nuestro DataFrame de Puno
todas_las_columnas = df_puno.columns.tolist()

# Buscamos columnas que contengan las palabras clave 'monto', 'inversion', 'actualizado'
columnas_relevantes = [col for col in todas_las_columnas if 'monto' in col and ('inversion' in col or 'actualizado' in col)]

print("Posibles nombres para la columna 'Monto de Inversión Actualizado':")
print(columnas_relevantes)

Posibles nombres para la columna 'Monto de Inversión Actualizado':
['monto_viable_actualizado']


In [16]:
# Tarea 1.3: Definir la muestra de análisis (Corregida con el nombre de columna correcto)
print("\n--- Paso 3: Definición de la Muestra de Análisis (Sección 4.1 de la Tesis) ---")

# Filtramos para la muestra final usando el nombre de columna correcto
analysis_sample = df_puno[
    (df_puno['estado_de_ejecucion'] == 'Finalizado') &
    (df_puno['monto_viable/aprobado'] > 0) & 
    (df_puno['monto_viable_actualizado'].notna())
].copy()

print(f"La muestra final para el análisis, según los criterios de la tesis, contiene {len(analysis_sample)} registros.")

# Caracterización de la muestra por modalidad de ejecución
print("\nComposición de la muestra por modalidad de ejecución:")
# ¡CORREGIDO! Se eliminó el paréntesis extra al final.
print(analysis_sample['modalidad_de_ejecucion_de_la_obra'].value_counts()) 


--- Paso 3: Definición de la Muestra de Análisis (Sección 4.1 de la Tesis) ---
La muestra final para el análisis, según los criterios de la tesis, contiene 4737 registros.

Composición de la muestra por modalidad de ejecución:
modalidad_de_ejecucion_de_la_obra
Administración directa            3500
Contrata                          1088
Por núcleo ejecutor                144
Asociacion Pública Privada APP       3
Obras por impuestos                  2
Name: count, dtype: int64


In [17]:
# --- Refinamiento Metodológico: Aislar los grupos de comparación ---
print("--- Refinando la muestra para el análisis comparativo ---")

# Guardamos el número de registros antes del filtro
registros_antes = len(analysis_sample)

# Nos quedamos únicamente con las dos modalidades de interés para la tesis
modalidades_de_interes = ['Administracion Directa', 'Contrata']
analysis_sample = analysis_sample[analysis_sample['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)].copy()

print(f"Se han filtrado los datos, eliminando {registros_antes - len(analysis_sample)} registros de otras modalidades.")
print(f"La muestra final para la comparación contiene {len(analysis_sample)} registros.")
print("\nComposición final de la muestra:")
print(analysis_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

--- Refinando la muestra para el análisis comparativo ---
Se han filtrado los datos, eliminando 3649 registros de otras modalidades.
La muestra final para la comparación contiene 1088 registros.

Composición final de la muestra:
modalidad_de_ejecucion_de_la_obra
Contrata    1088
Name: count, dtype: int64


In [19]:
# --- Paso 4: Análisis del Cumplimiento Presupuestal (Objetivo Específico 1) ---

# 1. Ingeniería de Característica: Desviación Presupuestal Porcentual (DP%)
analysis_sample['desviacion_presupuestal_%'] = \
    ((analysis_sample['monto_viable_actualizado'] - analysis_sample['monto_viable/aprobado']) 
     / analysis_sample['monto_viable/aprobado']) * 100

print("\nIndicador 'desviacion_presupuestal_%' calculado.")

# 2. Separar los datos por modalidad
admin_directa_dp = analysis_sample[analysis_sample['modalidad_de_ejecucion_de_la_obra'] == 'Administracion Directa']['desviacion_presupuestal_%'].dropna()
contrata_dp = analysis_sample[analysis_sample['modalidad_de_ejecucion_de_la_obra'] == 'Contrata']['desviacion_presupuestal_%'].dropna()

# 3. Generar la Tabla 2 de la tesis
tabla_2_dp = pd.DataFrame({
    'Administración Directa': admin_directa_dp.describe(),
    'Contrata': contrata_dp.describe()
}).T[['count', 'mean', '50%', 'std', 'min', 'max']]

tabla_2_dp.rename(columns={'50%': 'Mediana (DP%)', 'count': 'N', 'mean': 'Media (DP%)', 
                           'std': 'Desv. Estándar', 'min': 'Mínimo', 'max': 'Máximo'}, inplace=True)
tabla_2_dp['N'] = tabla_2_dp['N'].astype(int)

print("\n--- Tabla 2: Estadísticas Descriptivas de la Desviación Presupuestal Porcentual (DP%) ---")
display(tabla_2_dp.round(2))

# 4. Prueba de normalidad
print("\nPrueba de Normalidad (Shapiro-Wilk):")
print(f"  P-valor para Administración Directa: {shapiro(admin_directa_dp).pvalue:.4e}")
print(f"  P-valor para Contrata: {shapiro(contrata_dp).pvalue:.4e}")
print("--> Conclusión: Los datos no son normales (p < 0.05).")

# 5. Prueba de hipótesis
u_stat, p_value_dp = mannwhitneyu(admin_directa_dp, contrata_dp, alternative='two-sided')
print("\nPrueba de Hipótesis (U de Mann-Whitney) para Desviación Presupuestal:")
print(f"  Estadístico U = {u_stat:.0f}")
print(f"  p-value = {p_value_dp:.4f}")
if p_value_dp < 0.05:
    print("--> Conclusión: Existe una diferencia estadísticamente significativa (p < 0.05).")
else:
    print("--> Conclusión: No existe una diferencia estadísticamente significativa (p >= 0.05).")


Indicador 'desviacion_presupuestal_%' calculado.

--- Tabla 2: Estadísticas Descriptivas de la Desviación Presupuestal Porcentual (DP%) ---


,N,Media (DP%),Mediana (DP%),Desv. Estándar,Mínimo,Máximo
Administración Directa,0,NaN,NaN,NaN,NaN,NaN
Contrata,1088,-100.0,-100.0,0.0,-100.0,-100.0



Prueba de Normalidad (Shapiro-Wilk):
  P-valor para Administración Directa: nan
  P-valor para Contrata: 1.0000e+00
--> Conclusión: Los datos no son normales (p < 0.05).

Prueba de Hipótesis (U de Mann-Whitney) para Desviación Presupuestal:
  Estadístico U = nan
  p-value = nan
--> Conclusión: No existe una diferencia estadísticamente significativa (p >= 0.05).


C:\Users\GAMERS\AppData\Local\Temp\ipykernel_11884\3989933158.py:29: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  print(f"  P-valor para Administración Directa: {shapiro(admin_directa_dp).pvalue:.4e}")
D:\SEMESTRE ACTUAL 2025-II\MACHINE LEARNING OPERATIONS - ML Ops\project-iris\iris_env\Lib\site-packages\scipy\stats\_axis_nan_policy.py:579: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
C:\Users\GAMERS\AppData\Local\Temp\ipykernel_11884\3989933158.py:34: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  u_stat, p_value_dp = mannwhitneyu(admin_directa_dp, contrata_dp, alternative='two-sided')


# Análisis Cuantitativo para Tesis: Obras Públicas en Puno (2013-2025)

**Autor de la Tesis:** Bach. Fabricio Rigoberto Rodriguez Ascarrunz
**Análisis de Datos por:** Ing. Jhon Wilber Ajata Ascarrunz

**Objetivo:** Este notebook reproduce el análisis cuantitativo descrito en el Capítulo IV de la tesis. Se parte de la base de datos limpia de INFOBRAS para generar las tablas de resultados y realizar las pruebas de hipótesis que comparan las modalidades de ejecución por Administración Directa y por Contrata, siguiendo estrictamente la metodología y los indicadores definidos en el instrumento de investigación.

In [20]:
# --- Paso 1 y 2: Configuración del Entorno y Carga de Datos ---
import pandas as pd
import numpy as np
from scipy.stats import shapiro, mannwhitneyu

print("Librerías importadas correctamente.")

try:
    # Cargamos el resultado de nuestro trabajo de limpieza anterior
    df_clean = pd.read_csv('../data/infobras_clean_v1.csv', low_memory=False)
    
    # Aseguramos que las columnas de fecha clave sean del tipo datetime para futuros análisis
    date_cols_to_convert = ['fecha_de_inicio_de_obra', 'fecha_de_finalizacion_real']
    for col in date_cols_to_convert:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
    
    print("\nDataset 'infobras_clean_v1.csv' cargado y preparado.")
    
    # Filtramos para obtener solo los registros de la región Puno
    df_puno = df_clean[df_clean['departamento'] == 'PUNO'].copy()
    print(f"Se han aislado {len(df_puno)} obras correspondientes a la región Puno.")

except FileNotFoundError:
    print("\nError Crítico: No se encontró el archivo 'infobras_clean_v1.csv' en la carpeta 'data/'.")
    print("Asegúrate de que el archivo exista en la ubicación correcta antes de continuar.")

Librerías importadas correctamente.

Dataset 'infobras_clean_v1.csv' cargado y preparado.
Se han aislado 10317 obras correspondientes a la región Puno.


In [23]:
# --- Paso 3: Definición y Refinamiento de la Muestra de Análisis (Sección 4.1) ---

# 1. Filtro inicial para obras finalizadas con datos válidos
# **USAREMOS 'monto_de_ejecucion_financiera_de_la_obra' COMO CRITERIO, YA QUE ES LA VARIABLE FINAL CORRECTA**
initial_sample = df_puno[
    (df_puno['estado_de_ejecucion'] == 'Finalizado') &
    (df_puno['monto_viable/aprobado'] > 0) & 
    (df_puno['monto_de_ejecucion_financiera_de_la_obra'].notna()) # <-- ¡LA CORRECCIÓN MÁS IMPORTANTE!
].copy()

print(f"Muestra inicial (obras finalizadas en Puno): {len(initial_sample)} registros.")

# 2. Refinamiento metodológico: Aislar solo los grupos de comparación de la tesis
modalidades_de_interes = ['Administracion Directa', 'Contrata']
analysis_sample = initial_sample[initial_sample['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)].copy()

print(f"Muestra refinada para comparación directa: {len(analysis_sample)} registros.")

# 3. Caracterización de la muestra final
print("\nComposición de la muestra final por modalidad:")
print(analysis_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

Muestra inicial (obras finalizadas en Puno): 411 registros.
Muestra refinada para comparación directa: 80 registros.

Composición de la muestra final por modalidad:
modalidad_de_ejecucion_de_la_obra
Contrata    80
Name: count, dtype: int64


In [24]:
# --- Paso 4: Análisis del Cumplimiento Presupuestal (Objetivo Específico 1) ---

# 1. Ingeniería de Característica: Desviación Presupuestal Porcentual (DP%)
# **USANDO LA VARIABLE CORRECTA PARA EL COSTO FINAL**
analysis_sample['desviacion_presupuestal_%'] = \
    ((analysis_sample['monto_de_ejecucion_financiera_de_la_obra'] - analysis_sample['monto_viable/aprobado']) 
     / analysis_sample['monto_viable/aprobado']) * 100

print("\nIndicador 'desviacion_presupuestal_%' calculado con las variables correctas.")

# 2. Separar los datos por modalidad
admin_directa_dp = analysis_sample[analysis_sample['modalidad_de_ejecucion_de_la_obra'] == 'Administracion Directa']['desviacion_presupuestal_%'].dropna()
contrata_dp = analysis_sample[analysis_sample['modalidad_de_ejecucion_de_la_obra'] == 'Contrata']['desviacion_presupuestal_%'].dropna()

# 3. Generar la Tabla 2 de la tesis
tabla_2_dp = pd.DataFrame({
    'Administración Directa': admin_directa_dp.describe(),
    'Contrata': contrata_dp.describe()
}).T[['count', 'mean', '50%', 'std', 'min', 'max']]

tabla_2_dp.rename(columns={'50%': 'Mediana (DP%)', 'count': 'N', 'mean': 'Media (DP%)', 
                           'std': 'Desv. Estándar', 'min': 'Mínimo', 'max': 'Máximo'}, inplace=True)
tabla_2_dp['N'] = tabla_2_dp['N'].astype(int)

print("\n--- Tabla 2: Estadísticas Descriptivas de la Desviación Presupuestal Porcentual (DP%) ---")
display(tabla_2_dp.round(2))

# 4. Prueba de normalidad
print("\nPrueba de Normalidad (Shapiro-Wilk):")
print(f"  P-valor para Administración Directa: {shapiro(admin_directa_dp).pvalue:.4e}")
print(f"  P-valor para Contrata: {shapiro(contrata_dp).pvalue:.4e}")
print("--> Conclusión: Los datos no son normales (p < 0.05).")

# 5. Prueba de hipótesis
u_stat, p_value_dp = mannwhitneyu(admin_directa_dp, contrata_dp, alternative='two-sided')
print("\nPrueba de Hipótesis (U de Mann-Whitney) para Desviación Presupuestal:")
print(f"  Estadístico U = {u_stat:.0f}")
print(f"  p-value = {p_value_dp:.4f}")
if p_value_dp < 0.05:
    print("--> Conclusión: Existe una diferencia estadísticamente significativa (p < 0.05).")
else:
    print("--> Conclusión: No existe una diferencia estadísticamente significativa (p >= 0.05).")


Indicador 'desviacion_presupuestal_%' calculado con las variables correctas.

--- Tabla 2: Estadísticas Descriptivas de la Desviación Presupuestal Porcentual (DP%) ---


,N,Media (DP%),Mediana (DP%),Desv. Estándar,Mínimo,Máximo
Administración Directa,0,NaN,NaN,NaN,NaN,NaN
Contrata,80,-7.24,-10.2,127.65,-100.0,1010.66



Prueba de Normalidad (Shapiro-Wilk):
  P-valor para Administración Directa: nan
  P-valor para Contrata: 3.0149e-16
--> Conclusión: Los datos no son normales (p < 0.05).

Prueba de Hipótesis (U de Mann-Whitney) para Desviación Presupuestal:
  Estadístico U = nan
  p-value = nan
--> Conclusión: No existe una diferencia estadísticamente significativa (p >= 0.05).


C:\Users\GAMERS\AppData\Local\Temp\ipykernel_11884\3763513936.py:30: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  print(f"  P-valor para Administración Directa: {shapiro(admin_directa_dp).pvalue:.4e}")
C:\Users\GAMERS\AppData\Local\Temp\ipykernel_11884\3763513936.py:35: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  u_stat, p_value_dp = mannwhitneyu(admin_directa_dp, contrata_dp, alternative='two-sided')


In [25]:
# --- Celda de Detective Definitiva: Interrogando a las Modalidades por Separado ---

print("--- ANÁLISIS PARA 'ADMINISTRACIÓN DIRECTA' en PUNO ---")
df_ad = df_puno[df_puno['modalidad_de_ejecucion_de_la_obra'] == 'Administracion Directa']
print(f"Total de obras por Adm. Directa: {len(df_ad)}")
# ¿Cuántas obras por Adm. Directa tienen un monto de ejecución?
print(f"Obras con monto de ejecución financiera: {df_ad['monto_de_ejecucion_financiera_de_la_obra'].notna().sum()}")
# ¿Qué estado tienen las que SÍ tienen monto de ejecución?
print("\nEstado de ejecución de las obras por Adm. Directa QUE SÍ TIENEN MONTO:")
print(df_ad[df_ad['monto_de_ejecucion_financiera_de_la_obra'].notna()]['estado_de_ejecucion'].value_counts())


print("\n\n--- ANÁLISIS PARA 'CONTRATA' en PUNO ---")
df_co = df_puno[df_puno['modalidad_de_ejecucion_de_la_obra'] == 'Contrata']
print(f"Total de obras por Contrata: {len(df_co)}")
# ¿Cuántas obras por Contrata tienen un monto de ejecución?
print(f"Obras con monto de ejecución financiera: {df_co['monto_de_ejecucion_financiera_de_la_obra'].notna().sum()}")
# ¿Qué estado tienen las que SÍ tienen monto de ejecución?
print("\nEstado de ejecución de las obras por Contrata QUE SÍ TIENEN MONTO:")
print(df_co[df_co['monto_de_ejecucion_financiera_de_la_obra'].notna()]['estado_de_ejecucion'].value_counts())

--- ANÁLISIS PARA 'ADMINISTRACIÓN DIRECTA' en PUNO ---
Total de obras por Adm. Directa: 0
Obras con monto de ejecución financiera: 0

Estado de ejecución de las obras por Adm. Directa QUE SÍ TIENEN MONTO:
Series([], Name: count, dtype: int64)


--- ANÁLISIS PARA 'CONTRATA' en PUNO ---
Total de obras por Contrata: 2311
Obras con monto de ejecución financiera: 122

Estado de ejecución de las obras por Contrata QUE SÍ TIENEN MONTO:
estado_de_ejecucion
Finalizado      105
En Ejecución     17
Name: count, dtype: int64


In [26]:
# --- Paso 1 y 2: Configuración, Carga y BLINDAJE de Datos ---
import pandas as pd
import numpy as np
from scipy.stats import shapiro, mannwhitneyu

print("Librerías importadas.")

try:
    df_clean = pd.read_csv('../data/infobras_clean_v1.csv', low_memory=False)
    
    # --- PASO DE BLINDAJE ---
    # Limpiamos espacios en blanco al inicio y al final de todas las columnas de texto.
    # Esta es la causa más común de errores de filtrado.
    for col in df_clean.select_dtypes(include=['object']).columns:
        df_clean[col] = df_clean[col].str.strip()
    
    # Aseguramos fechas
    date_cols_to_convert = ['fecha_de_inicio_de_obra', 'fecha_de_finalizacion_real']
    for col in date_cols_to_convert:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
    
    print("\nDataset 'infobras_clean_v1.csv' cargado y BLINDADO.")
    
    # Filtramos para Puno (ahora el filtro será robusto)
    df_puno = df_clean[df_clean['departamento'] == 'PUNO'].copy()
    print(f"Se han aislado {len(df_puno)} obras de Puno.")
    print("\nComposición por modalidad en Puno (DESPUÉS DEL BLINDAJE):")
    print(df_puno['modalidad_de_ejecucion_de_la_obra'].value_counts())


except FileNotFoundError:
    print("\nError Crítico: No se encontró 'infobras_clean_v1.csv'.")

Librerías importadas.

Dataset 'infobras_clean_v1.csv' cargado y BLINDADO.
Se han aislado 10317 obras de Puno.

Composición por modalidad en Puno (DESPUÉS DEL BLINDAJE):
modalidad_de_ejecucion_de_la_obra
Administración directa            7705
Contrata                          2311
Por núcleo ejecutor                281
Asociacion Pública Privada APP      13
Obras por impuestos                  4
Por convenio entre entidades         2
Con organismo internacional          1
Name: count, dtype: int64


In [27]:
# --- Paso 3: Definición de la Muestra de Análisis ---
initial_sample = df_puno[
    (df_puno['estado_de_ejecucion'] == 'Finalizado') &
    (df_puno['monto_viable/aprobado'] > 0) & 
    (df_puno['monto_de_ejecucion_financiera_de_la_obra'].notna())
].copy()

modalidades_de_interes = ['Administracion Directa', 'Contrata']
analysis_sample = initial_sample[initial_sample['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)].copy()

print(f"Muestra refinada para comparación directa: {len(analysis_sample)} registros.")
print("\nComposición de la muestra final por modalidad:")
print(analysis_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

Muestra refinada para comparación directa: 80 registros.

Composición de la muestra final por modalidad:
modalidad_de_ejecucion_de_la_obra
Contrata    80
Name: count, dtype: int64


In [28]:
# --- Paso 4: Análisis del Cumplimiento Presupuestal ---
analysis_sample['desviacion_presupuestal_%'] = \
    ((analysis_sample['monto_de_ejecucion_financiera_de_la_obra'] - analysis_sample['monto_viable/aprobado']) 
     / analysis_sample['monto_viable/aprobado']) * 100

admin_directa_dp = analysis_sample[analysis_sample['modalidad_de_ejecucion_de_la_obra'] == 'Administracion Directa']['desviacion_presupuestal_%'].dropna()
contrata_dp = analysis_sample[analysis_sample['modalidad_de_ejecucion_de_la_obra'] == 'Contrata']['desviacion_presupuestal_%'].dropna()

tabla_2_dp = pd.DataFrame({
    'Administración Directa': admin_directa_dp.describe(),
    'Contrata': contrata_dp.describe()
}).T[['count', 'mean', '50%', 'std', 'min', 'max']]
tabla_2_dp.rename(columns={'50%': 'Mediana (DP%)', 'count': 'N', 'mean': 'Media (DP%)', 
                           'std': 'Desv. Estándar', 'min': 'Mínimo', 'max': 'Máximo'}, inplace=True)
tabla_2_dp['N'] = tabla_2_dp['N'].astype(int)

print("\n--- Tabla 2: Estadísticas Descriptivas ---")
display(tabla_2_dp.round(2))

u_stat, p_value_dp = mannwhitneyu(admin_directa_dp, contrata_dp, alternative='two-sided')
print(f"\nResultado Prueba U de Mann-Whitney: p-value = {p_value_dp:.4f}")


--- Tabla 2: Estadísticas Descriptivas ---


,N,Media (DP%),Mediana (DP%),Desv. Estándar,Mínimo,Máximo
Administración Directa,0,NaN,NaN,NaN,NaN,NaN
Contrata,80,-7.24,-10.2,127.65,-100.0,1010.66



Resultado Prueba U de Mann-Whitney: p-value = nan


C:\Users\GAMERS\AppData\Local\Temp\ipykernel_11884\649113193.py:20: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  u_stat, p_value_dp = mannwhitneyu(admin_directa_dp, contrata_dp, alternative='two-sided')


In [29]:
# --- Paso 1 y 2: Configuración, Carga y BLINDAJE de Datos ---
import pandas as pd
import numpy as np
from scipy.stats import shapiro, mannwhitneyu

print("Librerías importadas.")

try:
    df_clean = pd.read_csv('../data/infobras_clean_v1.csv', low_memory=False)
    
    # --- PASO DE BLINDAJE ---
    # Limpiamos espacios en blanco al inicio y al final de todas las columnas de texto.
    # Esta es la causa más común de errores de filtrado.
    for col in df_clean.select_dtypes(include=['object']).columns:
        df_clean[col] = df_clean[col].str.strip()
    
    # Aseguramos fechas
    date_cols_to_convert = ['fecha_de_inicio_de_obra', 'fecha_de_finalizacion_real']
    for col in date_cols_to_convert:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
    
    print("\nDataset 'infobras_clean_v1.csv' cargado y BLINDADO.")
    
    # Filtramos para Puno (ahora el filtro será robusto)
    df_puno = df_clean[df_clean['departamento'] == 'PUNO'].copy()
    print(f"Se han aislado {len(df_puno)} obras de Puno.")
    print("\nComposición por modalidad en Puno (DESPUÉS DEL BLINDAJE):")
    print(df_puno['modalidad_de_ejecucion_de_la_obra'].value_counts())


except FileNotFoundError:
    print("\nError Crítico: No se encontró 'infobras_clean_v1.csv'.")

Librerías importadas.

Dataset 'infobras_clean_v1.csv' cargado y BLINDADO.
Se han aislado 10317 obras de Puno.

Composición por modalidad en Puno (DESPUÉS DEL BLINDAJE):
modalidad_de_ejecucion_de_la_obra
Administración directa            7705
Contrata                          2311
Por núcleo ejecutor                281
Asociacion Pública Privada APP      13
Obras por impuestos                  4
Por convenio entre entidades         2
Con organismo internacional          1
Name: count, dtype: int64


In [30]:
# --- Paso 3: Definición de la Muestra de Análisis ---
initial_sample = df_puno[
    (df_puno['estado_de_ejecucion'] == 'Finalizado') &
    (df_puno['monto_viable/aprobado'] > 0) & 
    (df_puno['monto_de_ejecucion_financiera_de_la_obra'].notna())
].copy()

modalidades_de_interes = ['Administracion Directa', 'Contrata']
analysis_sample = initial_sample[initial_sample['modalidad_de_ejecucion_de_la_obra'].isin(modalidades_de_interes)].copy()

print(f"Muestra refinada para comparación directa: {len(analysis_sample)} registros.")
print("\nComposición de la muestra final por modalidad:")
print(analysis_sample['modalidad_de_ejecucion_de_la_obra'].value_counts())

Muestra refinada para comparación directa: 80 registros.

Composición de la muestra final por modalidad:
modalidad_de_ejecucion_de_la_obra
Contrata    80
Name: count, dtype: int64


In [31]:
# --- Paso 4: Análisis del Cumplimiento Presupuestal ---
analysis_sample['desviacion_presupuestal_%'] = \
    ((analysis_sample['monto_de_ejecucion_financiera_de_la_obra'] - analysis_sample['monto_viable/aprobado']) 
     / analysis_sample['monto_viable/aprobado']) * 100

admin_directa_dp = analysis_sample[analysis_sample['modalidad_de_ejecucion_de_la_obra'] == 'Administracion Directa']['desviacion_presupuestal_%'].dropna()
contrata_dp = analysis_sample[analysis_sample['modalidad_de_ejecucion_de_la_obra'] == 'Contrata']['desviacion_presupuestal_%'].dropna()

tabla_2_dp = pd.DataFrame({
    'Administración Directa': admin_directa_dp.describe(),
    'Contrata': contrata_dp.describe()
}).T[['count', 'mean', '50%', 'std', 'min', 'max']]
tabla_2_dp.rename(columns={'50%': 'Mediana (DP%)', 'count': 'N', 'mean': 'Media (DP%)', 
                           'std': 'Desv. Estándar', 'min': 'Mínimo', 'max': 'Máximo'}, inplace=True)
tabla_2_dp['N'] = tabla_2_dp['N'].astype(int)

print("\n--- Tabla 2: Estadísticas Descriptivas ---")
display(tabla_2_dp.round(2))

u_stat, p_value_dp = mannwhitneyu(admin_directa_dp, contrata_dp, alternative='two-sided')
print(f"\nResultado Prueba U de Mann-Whitney: p-value = {p_value_dp:.4f}")


--- Tabla 2: Estadísticas Descriptivas ---


,N,Media (DP%),Mediana (DP%),Desv. Estándar,Mínimo,Máximo
Administración Directa,0,NaN,NaN,NaN,NaN,NaN
Contrata,80,-7.24,-10.2,127.65,-100.0,1010.66



Resultado Prueba U de Mann-Whitney: p-value = nan


C:\Users\GAMERS\AppData\Local\Temp\ipykernel_11884\649113193.py:20: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  u_stat, p_value_dp = mannwhitneyu(admin_directa_dp, contrata_dp, alternative='two-sided')


In [32]:
# --- Celda de Detective Definitiva: Interrogando a las Modalidades por Separado ---

print("--- ANÁLISIS PARA 'ADMINISTRACIÓN DIRECTA' en PUNO ---")
df_ad = df_puno[df_puno['modalidad_de_ejecucion_de_la_obra'] == 'Administracion Directa']
print(f"Total de obras por Adm. Directa: {len(df_ad)}")
# ¿Cuántas obras por Adm. Directa tienen un monto de ejecución?
print(f"Obras con monto de ejecución financiera: {df_ad['monto_de_ejecucion_financiera_de_la_obra'].notna().sum()}")
# ¿Qué estado tienen las que SÍ tienen monto de ejecución?
print("\nEstado de ejecución de las obras por Adm. Directa QUE SÍ TIENEN MONTO:")
print(df_ad[df_ad['monto_de_ejecucion_financiera_de_la_obra'].notna()]['estado_de_ejecucion'].value_counts())


print("\n\n--- ANÁLISIS PARA 'CONTRATA' en PUNO ---")
df_co = df_puno[df_puno['modalidad_de_ejecucion_de_la_obra'] == 'Contrata']
print(f"Total de obras por Contrata: {len(df_co)}")
# ¿Cuántas obras por Contrata tienen un monto de ejecución?
print(f"Obras con monto de ejecución financiera: {df_co['monto_de_ejecucion_financiera_de_la_obra'].notna().sum()}")
# ¿Qué estado tienen las que SÍ tienen monto de ejecución?
print("\nEstado de ejecución de las obras por Contrata QUE SÍ TIENEN MONTO:")
print(df_co[df_co['monto_de_ejecucion_financiera_de_la_obra'].notna()]['estado_de_ejecucion'].value_counts())

--- ANÁLISIS PARA 'ADMINISTRACIÓN DIRECTA' en PUNO ---
Total de obras por Adm. Directa: 0
Obras con monto de ejecución financiera: 0

Estado de ejecución de las obras por Adm. Directa QUE SÍ TIENEN MONTO:
Series([], Name: count, dtype: int64)


--- ANÁLISIS PARA 'CONTRATA' en PUNO ---
Total de obras por Contrata: 2311
Obras con monto de ejecución financiera: 122

Estado de ejecución de las obras por Contrata QUE SÍ TIENEN MONTO:
estado_de_ejecucion
Finalizado      105
En Ejecución     17
Name: count, dtype: int64
